In [ ]:
from ipywebrtc import CameraStream, ImageRecorder
from ipywidgets import Video, Image, Button, Layout, TwoByTwoLayout, IntText
from IPython.display import display 
from keras.preprocessing import image
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import json 

In [ ]:
# This section handles camera functions
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 'auto', 'height': 'auto'}
                       })

In [ ]:
# Keras model import for guesses. 
import tensorflow as tf
import tensorflow_hub as hub
import glob
model = tf.keras.models.load_model('plant.h5',custom_objects={'KerasLayer':hub.KerasLayer})
# model.summary()

In [ ]:
# Class names for model predictions
class_names=['Apple;Apple_scab', 'Apple;Black_rot', 'Apple;Cedar_apple_rust', 'Apple;healthy', 'Blueberry;healthy', 'Cherry;healthy', 'Cherry;Powdery_mildew', 'Corn;Cercospora_leaf_spot_Gray_leaf_spot', 'Corn;Common_rust', 'Corn;healthy', 'Corn;Northern_Leaf_Blight', 'Grape;Black_rot', 'Grape;healthy', 'Heart Leaf Philodendron;citrus_greening', 'Heart Leaf Philodendron;healthy', 'Orange;Citrus_greening', 'Peach;Bacterial_spot', 'Peach;healthy', 'Pepper bell;Bacterial_spot', 'Pepper bell;healthy', 'Polka Dot;healthy', 'Potato;healthy', 'Potato;Late_blight', 'Potato;Early_blight', 'Raspberry;healthy', 'Soybean;healthy', 'Squash;Powdery_mildew', 'Strawberry;healthy', 'Strawberry;Leaf_scorch' 'Tomato;Target_Spot', 'Tomato;Tomato_Yellow_Leaf_Curl_Virus', 'Tomato__;Tomato_mosaic_virus', 'Tomato_;Bacterial_spot', 'Tomato;Early_blight', 'Tomato;healthy', 'Tomato;Late_blight', 'Tomato;Leaf_Mold', 'Tomato;Septoria_leaf_spot']

In [ ]:
IMAGE_SIZE = (224, 224)
# image loading function
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=IMAGE_SIZE)
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [ ]:
# prediction method 

def do_predict(test_image):
    new_image = load_image(test_image)
    pred = model.predict(new_image)
    # print(pred[0])
    prediction_result = class_names[np.argmax(pred[0])]
    
#     print(pred[0])
#     print(pred[0][np.argmax(pred[0])])
    if pred[0][np.argmax(pred[0])] < 0.073:
        prediction_result = "Unknown"
        
    # print('prediction=',prediction_result)
    prediction_handler(prediction_result)
    save_prediction(prediction_result)


    img = image.load_img(test_image,target_size=IMAGE_SIZE)
    
def save_prediction(disease_t): 
    data = json.load(open("disease_count.json"))
    try:
        current_val = data[disease_t]
        data[disease_t] =current_val + 1
        # print(data[disease_t])
        json.dump( data, open( "disease_count.json", 'w' ) )
        plot_gen(data)
        
    except:
        return None

In [ ]:
# Opens up the camera stream, does some loading of the images
front_camera = CameraStream.facing_user(audio=False)
image_recorder = ImageRecorder(stream=front_camera)
file = open("test_plot.png", "rb")
image_data = file.read()
file.close()

In [ ]:
# Data for plotting

def plot_gen(disease_data):
    
    disease_data = json.load(open("disease_count.json"))
    plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
    for dict_keys, dict_values in disease_data.items():
        plt.plot(dict_keys,dict_values, 'o')
        
    plt.savefig('test_plot.png')
    plt.close()
    return None 




In [ ]:
import PIL.Image
import PIL.ImageFilter
import io
import os

# Method handles the click event from the user
def do_predict_button(self):
     try: 
        im = PIL.Image.open(io.BytesIO(image_recorder.image.value))
        rgb= im.convert('RGB')
        rgb.save("recorded.jpg")
        do_predict("recorded.jpg")
        os.remove("recorded.jpg")
     except: 
         update_placeholder("Please capture the image first")




In [ ]:
# Checks the correct items to display, formats it and spits it out
def prediction_handler(p_prediction): 
    prediction = p_prediction.split(';')
    if "Healthy".lower() in prediction[-1].lower():
        plant_name = prediction[0]
        plant_status = 'Healthy'
        plant_disease = 'NIL'
        plant_solution = 'NIL'
        
    else: 
        plant_name = prediction[0]
        if "Unknown" in plant_name:
            plant_status = "Unknown"
            plant_solution = "Take another picture"
            plant_disease = "NIL"
            # this is temporary
            plant_disease = prediction[-1]
        else:
            plant_status = "Unhealthy"
            plant_solution = 'Increase pH level of water'
            plant_disease = p_prediction.split(';')[-1]
    
    text = (("Species: {}\nStatus: {}\nDisease: {}\nSolution: {}" .format(plant_name, plant_status,plant_disease,plant_solution)))
    update_placeholder(text)


In [ ]:
# Updates the textbox when needed
def update_placeholder(text):
    extra.placeholder = text
    
    

In [ ]:
from ipywidgets import GridBox

# Handles the generation of the GUI
header  = widgets.HBox([image_recorder], layout=Layout(width='auto', grid_area='header'))
main    = widgets.HBox([front_camera], layout=Layout(width='auto', height="120%", grid_area='main'))
sidebar = Button(description='PREDICTION',layout=Layout(width='auto', grid_area='sidebar'))
extra = widgets.Textarea(placeholder = "Capture an Image to Start", disabled = True,
                 layout=Layout(width='auto',height = '150%', grid_area='ex'))
footer  = widgets.Image(value=image_data, format='png', width='auto', height='auto',layout=Layout(width='auto', grid_area='footer'))
arduino_display = widgets.Textarea(placeholder="Arduino data", disabled=True,
                                   layout=Layout(width="auto",height="150%",grid_area="ad"))
arduino_header = widgets.HTML(value="<p style='text-align:center;'><b>Arduino Data</b></p>", layout=Layout(width="auto",height="auto",grid_area="ah"))


sidebar.on_click(do_predict_button)
GridBox(children=[header, main, sidebar, footer, extra, arduino_display, arduino_header],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "main main header header"
            "ah ah sidebar sidebar"
            "ad ad ex ex "
            "footer footer footer footer"
            ''')
       )

GridBox(children=(HBox(children=(ImageRecorder(image=Image(value=b''), stream=CameraStream(constraints={'audio…